# Segmenting and Clustering Neighborhoods in Toronto
-----------

### Import libraries

In [1]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

### Part1: Wikipedia page Scraping HTML Tables 

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name.strip('\n'),[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighbourhood
"


In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content().strip('\n') 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
[len(C) for (title,C) in col]

[181, 181, 181]

### Creating Dataframe from data scrapped

In [7]:
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Dataframe cleaning

In [8]:
# drop last row
df = df.drop(df.index[-1])

#Rename column 'Neighbourhood'
df.rename(columns = {'Neighbourhood' : 'Neighborhood'}, inplace = True)

In [9]:
# drop rows with 'Not assigned' in Borough's column
df1 =df[df['Borough'] != 'Not assigned']

In [10]:
df1.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
# Verify that column 'Neighborhood' has not 'Not assigned' value
df1[df1['Neighborhood'] == 'Not assigned']

,Postal Code,Borough,Neighborhood


In [12]:
# Number of rows
print("Number of rows in the dataframe: ",df1.shape[0])

Number of rows in the dataframe:  103


## Part 2: Get the latitude and the longitude coordinates of each neighborhood. 

Geocoder Python package does not work, so I download the csv file

In [13]:
GeoCoord = pd.read_csv("http://cocl.us/Geospatial_data")
GeoCoord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
GeoCoord.shape

(103, 3)

Merge the 2 dataframes to have only one with latitude and longitude features of each postal code

In [15]:
df_Toronto = df1.merge(GeoCoord,on='Postal Code')
df_Toronto.shape

(103, 5)

In [16]:
df_Toronto.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [17]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_Toronto['Borough'].unique()),
        df_Toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.
